In [1]:
import os
import pandas as pd
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
location='../data/train'
classes=[]
videos=[]
train=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
train['video_name']=videos
print(classes)
train.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
1,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
2,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
3,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
4,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...


In [4]:
location='../data/test'
classes=[]
videos=[]
test=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
test['video_name']=videos
print(classes)
test.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
1,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
2,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
3,brush_hair/Blonde_being_brushed_brush_hair_f_n...
4,brush_hair/Blonde_being_brushed_brush_hair_u_c...


In [5]:
location='../data/val'
classes=[]
videos=[]
val=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
val['video_name']=videos
print(classes)
val.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UP...
1,brush_hair/Brushing_hair__the_right_way_brush_...
2,brush_hair/Brushing_my_Long_Hair__February_200...
3,brush_hair/Ella_brushing_her_amazing_long_hair...
4,brush_hair/Haarek_mmen_brush_hair_h_cm_np1_fr_...


In [6]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

In [7]:
# creating tags for test videos
val_video_tag = []
for i in range(val.shape[0]):
    val_video_tag.append(val['video_name'][i].split('/')[0])
    
val['tag'] = val_video_tag

In [8]:
# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

In [9]:
train_data = pd.DataFrame(columns=['image','class'])
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    classname=videoFile.split(' ')[0].split('/')[0]
#    print('../data/train/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])
    cap = cv2.VideoCapture('../data/train/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            name=(videoFile.split('/')[1]).split('.')[0];
            filename ='../data/train_frame/' + name +"_frame%d.jpg" % count;
            imgname=name +"_frame%d.jpg" % count            
            train_data=train_data.append({'image':imgname,'class':classname},ignore_index=True)
#            print(filename)
            cv2.imwrite(filename, frame)
            count+=1
    cap.release()

100%|██████████| 3570/3570 [07:45<00:00,  8.41it/s]


In [10]:
train_data.head()

,image,class
0,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
1,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
2,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
3,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
4,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair


In [11]:
train_data.nunique()

image    13022
class       51
dtype: int64

In [12]:
# converting the dataframe into csv file 
train_data.to_csv('../data/train.csv',header=True, index=False)

In [13]:
train = pd.read_csv('../data/train.csv')
train.head()

,image,class
0,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
1,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
2,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
3,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
4,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair


In [14]:
val_data = pd.DataFrame(columns=['image','class'])
# storing the frames from training videos
for i in tqdm(range(val.shape[0])):
    count = 0
    videoFile = val['video_name'][i]
    classname=videoFile.split(' ')[0].split('/')[0]
#    print('../data/train/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])
    cap = cv2.VideoCapture('../data/val/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            name=(videoFile.split('/')[1]).split('.')[0];
            filename ='../data/val_frame/' + name +"_frame%d.jpg" % count;
            imgname=name +"_frame%d.jpg" % count            
            val_data=val_data.append({'image':imgname,'class':classname},ignore_index=True)
#            print(frame)
            cv2.imwrite(filename, frame)
            count+=1
    cap.release()

100%|██████████| 1666/1666 [03:24<00:00,  5.22it/s]


In [15]:
val_data.nunique()

image    5509
class      51
dtype: int64

In [16]:
# converting the dataframe into csv file 
val_data.to_csv('../data/val.csv',header=True, index=False)

In [17]:
val = pd.read_csv('../data/val.csv')
val.head()

,image,class
0,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
1,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
2,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
3,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
4,Brushing_hair__the_right_way_brush_hair_u_nm_n...,brush_hair
